In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executer.memory", "4g")
    .getOrCreate()
)

22/09/18 16:43:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/18 16:43:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
sdf = spark.read.parquet("../data/curated/full_data/")
sdf.count()

7817737

In [4]:
sdf.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- SA2_code: integer (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- consumer_id: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- mean_total_income: integer (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- take_rate: double (nullable = true)



In [5]:
sdf.show(1, truncate=False, vertical=True)

-RECORD 0-------------------------------------------------
 user_id           | 183                                  
 SA2_code          | 210051245                            
 postcode          | 3043                                 
 consumer_id       | 133032                               
 state             | VIC                                  
 gender            | Female                               
 mean_total_income | 58060                                
 merchant_abn      | 38700038932                          
 dollar_value      | 1115.1879869740476                   
 order_id          | d4432c96-0627-4034-8934-1f745277f6bc 
 name              | Etiam Bibendum Industries            
 tags              | tent                                 
 revenue_level     | a                                    
 take_rate         | 6.31                                 
only showing top 1 row



In [6]:
for col in sdf.columns:
    print(f"Number of missiong values in {col}:", sdf.filter(F.col(col).isNull()).count())

Number of missiong values in user_id: 7
Number of missiong values in SA2_code: 5208
Number of missiong values in postcode: 7
Number of missiong values in consumer_id: 7
Number of missiong values in state: 7
Number of missiong values in gender: 7
Number of missiong values in mean_total_income: 7
Number of missiong values in merchant_abn: 0
Number of missiong values in dollar_value: 7
Number of missiong values in order_id: 7
Number of missiong values in name: 0
Number of missiong values in tags: 0
Number of missiong values in revenue_level: 0
Number of missiong values in take_rate: 0


In [7]:
missing = sdf.filter(F.col("SA2_code").isNull()).groupBy("postcode").count()
missing.count() # 2 postcodes does not have a corresponding SA2 code
missing

postcode,count
null,7
6958,3242
3989,1959


In [ ]:
#null and missing values
'''
consumer merge with total_income:
  missing value: mean_total_income for some regions (44 rows with missing values)
  => use mean_total_income of state to replace null (ETL.py line51)

continue to merge with merchant and transaction:
  postcode = 6958 and 3989 have missing SA2
  number of rows affected: 5644, 3376
  we checked this two place and they are very secluded, therefore we left their SA2s as missing


'''


In [ ]:
#outlier
'''after joining everything as one dataframe, 
   we filtered rows with amount of transaction that are outside the quartile range(0.025---0.975) per tag'''


